In [ ]:
!pip install chromadb sentence-transformers boto3

In [ ]:
access_key_id = ""
secret_key = ""

In [ ]:
import boto3
import json

# Initialize the Bedrock Runtime client
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",  # Replace with your AWS region
    aws_access_key_id=access_key_id,  # Replace with your AWS access key
    aws_secret_access_key=secret_key  # Replace with your AWS secret key
)

#### Decline Request

In [ ]:
# Define the messages as per ChatML format
system_prompt = [
    {"text": "You are a current affairs expert. Provide user with specific information he asks for. Only answer related to current affairs other wise politely decline. Do not generate essays or poems"}
]

# Define the user message
messages = [
    {
        "role": "user",
        "content": [{"text": "Write a short poem about spring."}]
    }
]

body = json.dumps({
    "schemaVersion": "messages-v1",
    "system": system_prompt,
    "messages": messages,
    "inferenceConfig": {
        "maxTokens": 500,
        "temperature": 0.7,
        "topP": 0.9
    }
})

# Construct the body as JSON
# body = json.dumps({
#     "messages": messages

# })

# Model ID for Amazon Nova Lite
model_id = "amazon.nova-lite-v1:0"

# Invoke the model
response = bedrock_client.invoke_model(
    modelId=model_id,
    body=body,
    contentType="application/json",
    accept="application/json"
)

# Parse the response
response_body = json.loads(response["body"].read())


In [ ]:
print(response_body['output']['message']['content'][0]['text'])

#### Simple Question, Outdated information

In [ ]:
messages = [
    {
        "role": "user",
        "content": [{"text": ""}]
    }
]

body = json.dumps({
    "schemaVersion": "messages-v1",
    "system": system_prompt,
    "messages": messages,
    "inferenceConfig": {
        "maxTokens": 500,
        "temperature": 0.7,
        "topP": 0.9
    }
})

# Construct the body as JSON
# body = json.dumps({
#     "messages": messages

# })

# Model ID for Amazon Nova Lite
model_id = "amazon.nova-lite-v1:0"

# Invoke the model
response = bedrock_client.invoke_model(
    modelId=model_id,
    body=body,
    contentType="application/json",
    accept="application/json"
)

# Parse the response
response_body = json.loads(response["body"].read())


In [ ]:
print(response_body['output']['message']['content'][0]['text'])

In [ ]:
messages = [
    {
        "role": "user",
        "content": [{"text": ""}]
    }
]

body = json.dumps({
    "schemaVersion": "messages-v1",
    "system": system_prompt,
    "messages": messages,
    "inferenceConfig": {
        "maxTokens": 500,
        "temperature": 0.7,
        "topP": 0.9
    }
})

# Construct the body as JSON
# body = json.dumps({
#     "messages": messages

# })

# Model ID for Amazon Nova Lite
model_id = "amazon.nova-lite-v1:0"

# Invoke the model
response = bedrock_client.invoke_model(
    modelId=model_id,
    body=body,
    contentType="application/json",
    accept="application/json"
)

# Parse the response
response_body = json.loads(response["body"].read())


In [ ]:
print(response_body['output']['message']['content'][0]['text'])

#### Initializing Vector DB

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

# Initialize ChromaDB client
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="qaCollection")

# Load the embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Sample documents
documents = [
    {"id": "1", "text": ""},
    {"id": "2", "text": ""},
    {"id": "3", "text": ""}
]

# Add documents to ChromaDB
for doc in documents:
    embedding = embedding_model.encode(doc["text"]).tolist()
    collection.add(
        ids=[doc["id"]],
        embeddings=[embedding],
        documents=[doc["text"]]
    )


#### Define Question Answering Function utilizing RAG

In [ ]:


def answer_question(question):
    # Generate embedding for the question
    question_embedding = embedding_model.encode(question).tolist()

    # Retrieve relevant documents from ChromaDB
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=1
    )

    # Extract the most relevant document
    if results['documents']:
        context = results['documents'][0][0]
    else:
        context = ""

    # Construct the prompt for the Nova Lite model
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"

    # Prepare the request body

    messages = [
    {
        "role": "user",
        "content": [{"text": prompt}]
    }
    ]
    body = json.dumps({
    "schemaVersion": "messages-v1",
    "system": system_prompt,
    "messages": messages,
    "inferenceConfig": {
        "maxTokens": 5000,
        "temperature": 0.7,
        "topP": 0.9
    }
})

    # Invoke the Nova Lite model
    response = bedrock_client.invoke_model(
        modelId="amazon.nova-lite-v1:0",  # Replace with the correct model ID
        body=body,
        contentType="application/json",
        accept="application/json"
    )

    # Parse and return the response
    response_body = json.loads(response['body'].read())
    return response_body['output']['message']['content'][0]['text'], context


In [ ]:
question = ""
response, context = answer_question(question)

In [ ]:
print(response)

In [ ]:
print(context)

In [ ]:
question = ""
response, context = answer_question(question)

In [ ]:
print(response)

In [ ]:
print(context)